# mBuild Tutorial 08: Building Polymers

This tutorial demonstrates how to build polymers using the built-in `Polymer` class.

As in the prior tutorials, we need to first import mbuild (here as `mb`).

In [1]:
import warnings
warnings.filterwarnings('ignore')

import mbuild as mb

from mbuild.lib.recipes.polymer import Polymer



In the prior tutorials we created a class for constructing a linear alkane chain and also used a similar built-in `Alkane` class. One could imagine that the same approach we took to create this class (i.e. successively adding CH2 units) could be further generalized to support the creation of any linear polymer. In fact, mBuild contains a class that does just this, `mbuild.Polymer`.

Here, we'll explore how `Polymer` works by creating the familiar linear alkane, along with a PEG (polyethylene glycol) molecule. 

To be flexible, the `Polymer` class accepts, as arguments, repeat units (defined as `Compounds`), thus we will first define `Compound` classes for our CH2 and oxygen monomer units (with the appropriate `Ports`).

In [2]:
class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        
        mb.load('utils/ch2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        up_port = mb.Port(anchor=carbon, orientation=[0, 0, 1], separation=0.075)
        down_port = mb.Port(anchor=carbon, orientation=[0, 0, -1], separation=0.075)
        self.add(up_port, label='up')
        self.add(down_port, label='down')

class O(mb.Compound):
    def __init__(self):
        super(O, self).__init__()
        
        self.add(mb.Particle(name='O'))
        up_port = mb.Port(anchor=self[0], orientation=[0, 0, 1], separation=0.075)
        self.add(up_port, 'up')
        down_port = mb.Port(anchor=self[0], orientation=[0, 0, -1], separation=0.075)
        self.add(down_port, 'down')

To create a simple linear alkane, we would need to simply pass CH2 as the monomer. 
- `n` defines the number of times this is repeated. 

In [3]:
alkane = mb.recipes.Polymer(monomers=[CH2()])
alkane.build(n=6)

In [4]:
alkane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

To create PEG, we will now feed instances of both of the monomers (i.e. `CH2` and `O`) to the `monomers` argument of the `Polymer` class. 

We also need to provide a few additional arguments. 
- `sequence` provides a string of characters where each unique character represents one repetition of a monomer. Here, `AAB` means that we want two `CH2`'s for each `O` (`CH2` is defined as `A` because it is passed first, `O` is `B` as it is second).  
- As before, the `n` argument specifies the number of repeat units, however in this case, it is how many times to repeat the pattern.  That is, `n=1` would define a two `CH2` groups and an `O`.  

In [5]:
peg4 = mb.recipes.Polymer(monomers=[CH2(), O()])
peg4.build(sequence='AAB', n=4,)
peg4.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

The sequence argument can be defined with any arbitrary pattern.  For example, while not realistic from a chemistry standpoint, one could trivially create a rather complex sequence, such as `AABAABBBB`.

In [6]:
poly = mb.recipes.Polymer(monomers=[CH2(), O()])
poly.build(sequence='AABAABBBB', n=3)
poly.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Let us now consider creating a semifluorinated alkane using the polymer class (i.e., one half is an alkane, R_H, the other half is a perfluroalkane, R_F).  We first need to define a CF2 group, in the same way we defined CH2.

In [7]:
class CF2(mb.Compound):
    def __init__(self):
        super(CF2, self).__init__()
        
        mb.load('utils/cf2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        up_port = mb.Port(anchor=carbon, orientation=[0, 0, 1], separation=0.075)
        down_port = mb.Port(anchor=carbon, orientation=[0, 0, -1], separation=0.075)
        self.add(up_port, label='up')
        self.add(down_port, label='down')


CF2().visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

There are multiple ways to create the semifluorinated alkane; here we will consider two different approaches that make use of polymer class.  First, we simply define the sequence to describe the entire polymer, setting `n=1`.

In [8]:
semifluorinated_hexane = mb.recipes.Polymer(monomers=[CH2(), CF2()])
semifluorinated_hexane.build(sequence='AAABBB', n=1,)
semifluorinated_hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

The approach above works fine, but requires changing the sequence argument to adjust the length of each block in the polymer.  

Alternatively, we can take advantage of the flexible nature of mBuild, in that the `Polymer` class outputs `Compounds` and can accept as input `Compounds` (i.e., it is not limited to simple monomers). 

As such, let us call the `Polymer` class 3 times
- create the alkane block
- create the fluorinated block
- stitch the two blocks together


In [9]:
alkane_block = mb.recipes.Polymer(monomers=[CH2()])
alkane_block.build(n=3,)

pfa_block = mb.recipes.Polymer(monomers=[CF2()])
pfa_block.build(n=3)

In [10]:
semifluorinated_hexane = mb.recipes.Polymer()
semifluorinated_hexane.add_monomer(compound=alkane_block, indices=[10, 9])
semifluorinated_hexane.add_monomer(compound=pfa_block, indices=[10, 9])

semifluorinated_hexane.build(n=3,sequence='AB')
semifluorinated_hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

This of course can be wrapped in a flexible class that takes the length of each block as an argument, similar to what has been done in prior tutorials.  

In [11]:
class Semifluorinated(mb.Compound):
    def __init__(self, alkane_length, pfa_length):
        super(Semifluorinated, self).__init__()
        
        
        alkane_block = mb.recipes.Polymer(monomers=[CH2()])
        alkane_block.build(n=alkane_length,)

        pfa_block = mb.recipes.Polymer(monomers=[CF2()])
        pfa_block.build(n=pfa_length)

        semifluorinated_hexane = mb.recipes.Polymer()
        semifluorinated_hexane.add_monomer(compound=alkane_block, indices=[(3*alkane_length) + 1, 3*alkane_length])
        semifluorinated_hexane.add_monomer(compound=pfa_block, indices=[(3*pfa_length) + 1, 3*pfa_length])

        semifluorinated_hexane.build(n=3, sequence='AB')
        self.add(semifluorinated_hexane)

semifluorinated_alkane = Semifluorinated(alkane_length=5, pfa_length=3)
semifluorinated_alkane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Recap

The goal of this tutorial was to demonstrate how to use mBuild's `Polymer` class to create polymers with user-defined sequences.

The next tutorial will teach you how you can use mBuild to functionalize surfaces.